In [190]:
import os
import pandas as pd
import numpy as np
import io

#this bit all depends on what dictionary you're using
direct = #directory

if os.getcwd() != direct: 
    os.chdir(direct)

from_dir = [i for i in os.listdir() if ('american' in i) and ('abbreviation' not in i) and ('proper' not in i)]
from_dir += ['words.txt']
#from_dir = [i for i in os.listdir()]

def word_scrape(in_list): 
    outlist = []
    for i in in_list: 
        with open(i,'r') as my_file: 
            #print(i)
            for line in my_file: 
                #print(line)
                outlist.append(line)
    return outlist

startlist = word_scrape(from_dir)


uselist = [i[:-1] for i in startlist if "\'" not in i and '.' not in i and '-' not in i and i != '']
def remove_badletters(inlist,letters):
    df = pd.DataFrame(inlist)
    df = df.rename(columns={0:'words'})
    alphabet = 'abcdefghijklmnopqrstuvwxyz-1234567890ABCDEFGHIJLKMNOPQRSTUVWXYZ'
    letters = [i for i in alphabet if i in letters]
    badletters = [i for i in alphabet if i not in letters]
    for i in alphabet: 
        df[i] = df['words'].str.contains(i).astype(int)
    df = df.fillna(0)
    
    df['sum_bad_letters'] = df[badletters].sum(axis=1)
    df['sum_bad_letters'] = df['sum_bad_letters']!=0
    df = df.drop(df[df['sum_bad_letters']!=0].index)
    df = df.drop(['sum_bad_letters'],axis=1).reset_index(drop=True)
    return df
    
    
df = remove_badletters(uselist,'above')


In [183]:
class squaredlesquare():
    def __init__(self, letter = None, coordinate = None):
        self.letter = letter
        self.coordinate = coordinate
    def get_letter(self):
        return self.letter
    def get_coordinate(self):
        return self.coordinate
    def get_surroundings(self): 
        outlist = []
        for i in [-1,0,1]: 
            for j in [-1,0,1]: 
                x_coord = self.coordinate[0] + i
                y_coord = self.coordinate[1] + j
                if x_coord >= 0 and y_coord >= 0 and (x_coord,y_coord) != self.coordinate:
                    outlist.append((x_coord,y_coord))
        return outlist

class squaredle():
    def __init__(self, layout = []):
        self.layout = layout
        self.map_layout = [[(j,i) for i in range(len(self.layout[j]))] for j in range(len(self.layout))]
        self.row_size = len(self.layout)
        if self.row_size >= 1:
            self.col_size = len(self.layout[0])
        else: 
            self.col_size = 0
        total_squares = []
        for rowdex in range(len(layout)): 
            row_add = []
            row_letter = self.layout[rowdex]
            row_map = self.map_layout[rowdex]
            for coldex in range(len(row_letter)):
                letter = row_letter[coldex]
                map = row_map[coldex]
                row_add.append(squaredlesquare(letter,map))
            total_squares.append(row_add)
        self.total_squares = total_squares   
    
    def __str__(self): 
        return str(self.layout) + str(self.map_layout)

    def __getitem__(self,coord): 
        return self.total_squares[coord[0]][coord[1]]
    
    def prettyprint(self,store=False): 
        rowlist = [''.join(row) for row in self.layout]
        prettylist = '\n'.join(rowlist)
        print(prettylist)
        if store:
            return prettylist
    
    def get_coord_surroundings(self,coord):
        use_square = self.total_squares[coord[0]][coord[1]]
        raw_surroundings = use_square.get_surroundings()
        surroundings = [i for i in raw_surroundings if i[0] < self.row_size and i[1] < self.col_size]
        return surroundings

    def worm_out(self,coord,finished = [],in_usestr = '',goodlist = []): 
        donelist = finished
        outlist = []
        square = self[coord]
        coordinate = square.get_coordinate()
        usestr = in_usestr + square.get_letter()

        #print(f"Testing {usestr}")
        
        okay_words = []
        for word in goodlist: 
            if len(word) >= 4 and word == usestr: 
                outlist.append(word)
            elif word[0:len(usestr)] == usestr:
                okay_words.append(word)
        #print(okay_words[0:5])
        
        if okay_words == []: 
            return []
        
        surroundings = self.get_coord_surroundings(coordinate)
        #print(f"Original surrounding letters: {[self[i].get_letter() for i in surroundings]}")
        surroundings = [i for i in surroundings if i not in donelist]
        #print(f"Surrounding letters: {[self[i].get_letter() for i in surroundings]}")
        
        for surrounding in surroundings: 
            new_finished = donelist + [coordinate]
            #print(f"List of finished letters: {[self[c].get_letter() for c in new_finished]}") 
            wormed_words = self.worm_out(surrounding,new_finished,usestr,okay_words)
            outlist += wormed_words
            
        return outlist
    
    def get_all_worms(self,goodlist = []): 
        good_words = []
        for row in self.map_layout: 
            for coord in row: 
                curr_words = self.worm_out(coord,[],'',goodlist)
                good_words += curr_words
        return list(set(good_words))
    
        
        


In [191]:
test_squaredle = squaredle([['w','a','k','s'],['a','y','e','i'],['g','l','x','b'],['k','r','a','f']])
test_squaredle.prettyprint()

waks
ayei
glxb
kraf


In [192]:
test_squaredle[(0,1)].get_letter()
test_squaredle[(0,1)].get_coordinate()

(0, 1)

In [193]:
test_squaredle.get_all_worms(uselist)

['bise',
 'agla',
 'glar',
 'farle',
 'arle',
 'sike',
 'bales',
 'away',
 'walk',
 'gley',
 'skywalk',
 'belga',
 'leak',
 'gales',
 'xyla',
 'ales',
 'kebar',
 'waes',
 'axles',
 'beak',
 'seaway',
 'barkle',
 'seak',
 'fala',
 'argal',
 'gleba',
 'bike',
 'grab',
 'skel',
 'yelk',
 'rale',
 'belar',
 'axle',
 'yaks',
 'galyak',
 'agly',
 'wyle',
 'alga',
 'barley',
 'bark',
 'belay',
 'wale',
 'balei',
 'skey',
 'kibe',
 'galea',
 'ibex',
 'balk',
 'argle',
 'awake',
 'skylark']

In [91]:
test_something = [i for i in uselist if i == 'bake']